<a href="https://colab.research.google.com/github/prateek0489/100daysToDS/blob/master/FirstNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# # import libraries
# import fastai
# from fastai import *
# from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import string
import io
import os
import re
pd.set_option('display.max_colwidth', 100)


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

In [0]:
training_data=pd.read_csv("train.csv")

In [16]:
training_data.tweet_contains_url.value_counts()

1    4253
0    3562
2      91
3      10
4       3
5       1
Name: tweet_contains_url, dtype: int64

In [0]:
training_data['tweet_contains_url'] = training_data['tweet'].apply(lambda x: len(re.findall("(?P<url>https?://[^\s]+)", x)))
training_data['tweet_contains_smiley'] =training_data['tweet'].apply(lambda x: len(re.findall("((:|;)\))+", x)))

In [15]:
training_data.groupby('label').sum()
#agg({'tweet_contains_url':sum})

,id,tweet_contains_url,tweet_contains_smiley
label,,,
0,23307807,4276,215
1,8059353,206,16


In [19]:
len(training_data[(training_data.label==0)&(training_data.tweet_contains_url==0)])

1740

In [20]:
len(training_data[(training_data.label==0)&(training_data.tweet_contains_url==1)])

4050

In [21]:
len(training_data[(training_data.label==1)&(training_data.tweet_contains_url==0)])

1822

In [22]:
len(training_data[(training_data.label==1)&(training_data.tweet_contains_url==1)])

203

In [85]:
training_data['tweet_contains_smiley'].corr(training_data['label'])

-0.06959690708945807

In [0]:
def clean_text(text):
    text = re.sub("(?P<url>https?://[^\s]+)","",text)
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopwords]
    return text

In [83]:
training_data.head(20)

,id,label,tweet,tweet_contains_url,tweet_contains_smiley
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,1,0
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/,1,1
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu,1,0
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/,1,1
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!,0,0
5,6,1,iPhone software update fucked up my phone big time Stupid iPhones,0,0
6,7,0,Happy for us .. #instapic #instadaily #us #sony #xperia #xperiaZ https://instagram.com/p/z9qGfWlvj7/,1,0
7,8,0,New Type C charger cable #UK http://www.ebay.co.uk/itm/-/112598674021 … #bay #Amazon #etsy New Year #Rob Cross #Toby Young #EVEMUN #McMafia #Taylor #SPECTRE 2018 #NewYear #Starting 2018 #recipes #...,1,0
8,9,0,Bout to go shopping again listening to music #iphone #justme #music #likeforlike #followforfollow… http://instagr.am/p/Vj6bg5tLql/,1,0
9,10,0,Photo: #fun #selfie #pool #water #sony #camera #picoftheday #sun #instagood #boy #cute #outdoor... http://tmblr.co/ZAjC0n1ms_Gp8,1,0


In [0]:
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopwords]
    return text

training_data['body_text_nostop'] = training_data['tweet'].apply(lambda x: clean_text(x.lower()))

training_data.head()

,id,label,tweet,body_text_nostop
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,"[fingerprint, pregnancy, test, httpsgooglh1mfq..."
1,2,0,Finally a transparant silicon case ^^ Thanks t...,"[finally, transparant, silicon, case, thanks, ..."
2,3,0,We love this! Would you go? #talk #makememorie...,"[love, would, go, talk, makememories, unplug, ..."
3,4,0,I'm wired I know I'm George I was made that wa...,"[im, wired, know, im, george, made, way, iphon..."
4,5,1,What amazing service! Apple won't even talk to...,"[amazing, service, apple, wont, even, talk, qu..."


In [0]:
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()
def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

training_data['body_text_lemmatized'] = training_data['body_text_nostop'].apply(lambda x: lemmatizing(x))

training_data.head(10)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,id,label,tweet,body_text_nostop,body_text_lemmatized
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,"[fingerprint, pregnancy, test, httpsgooglh1mfq...","[fingerprint, pregnancy, test, httpsgooglh1mfq..."
1,2,0,Finally a transparant silicon case ^^ Thanks t...,"[finally, transparant, silicon, case, thanks, ...","[finally, transparant, silicon, case, thanks, ..."
2,3,0,We love this! Would you go? #talk #makememorie...,"[love, would, go, talk, makememories, unplug, ...","[love, would, go, talk, makememories, unplug, ..."
3,4,0,I'm wired I know I'm George I was made that wa...,"[im, wired, know, im, george, made, way, iphon...","[im, wired, know, im, george, made, way, iphon..."
4,5,1,What amazing service! Apple won't even talk to...,"[amazing, service, apple, wont, even, talk, qu...","[amazing, service, apple, wont, even, talk, qu..."
5,6,1,iPhone software update fucked up my phone big ...,"[iphone, software, update, fucked, phone, big,...","[iphone, software, update, fucked, phone, big,..."
6,7,0,Happy for us .. #instapic #instadaily #us #son...,"[happy, us, instapic, instadaily, us, sony, xp...","[happy, u, instapic, instadaily, u, sony, xper..."
7,8,0,New Type C charger cable #UK http://www.ebay.c...,"[new, type, c, charger, cable, uk, httpwwwebay...","[new, type, c, charger, cable, uk, httpwwwebay..."
8,9,0,Bout to go shopping again listening to music #...,"[bout, go, shopping, listening, music, iphone,...","[bout, go, shopping, listening, music, iphone,..."
9,10,0,Photo: #fun #selfie #pool #water #sony #camera...,"[photo, fun, selfie, pool, water, sony, camera...","[photo, fun, selfie, pool, water, sony, camera..."
